# Collaborative filtering

- Coger subsets aleatorios
- Ver si usar item based collab porque tiene mas escalabilidad, user based necesita calcular la similitud de todos los pares de usuarios

## Data loading

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
RANDOM_STATE = 5482

In [2]:
#df_users = pd.read_csv('data/train_triplets.txt', sep='\t', header=None, names=['user_id', 'song_id', 'play_count'])
df_users_raw = pd.read_csv('data/Million Song Dataset kaggle/User Listening History.csv')

In [3]:
df_music = pd.read_csv('data/Million Song Dataset kaggle/Music Info.csv')

In [4]:
df_users_raw.dtypes

track_id     object
user_id      object
playcount     int64
dtype: object

In [5]:
df_users_raw.shape

(9711301, 3)

In [67]:
num_distinct_users_raw = df_users_raw['user_id'].nunique()
num_distinct_users_raw

962037

We only take into account users that have listened more than 5 songs

In [7]:
df_user_counts = df_users_raw['user_id'].value_counts().reset_index()
df_user_counts.columns = ['user_id', 'count']
df_user_counts

,user_id,count
0,ec6dfcf19485cb011e0b22637075037aae34cf26,784
1,4e11f45d732f4861772b2906f81a7d384552ad12,384
2,726da71c2c2ea119119a7957517fccd028d1be76,376
3,113255a012b2affeab62607563d03fbdf31b08e7,367
4,7adec7f006cb09482d36609d205293d8b61f030e,366
...,...,...
962032,19e70c61de7bbb0ca3fc39c81a7e6b7eede69aa1,1
962033,7154895e6b692062033eecb3fe59ad240bb02811,1
962034,5d563bc9f80ddb3bce489c84b453409a88bd37e9,1
962035,779074ba35bd5f14913635819233a01dc5ecc9de,1


In [8]:
df_users = df_users_raw[df_users_raw['user_id'].isin(df_user_counts[df_user_counts['count'] > 5]['user_id'])].reset_index(drop=True)
#df_users = df_users_raw
df_users.shape

(8332242, 3)

In [69]:
num_distinct_users = df_users['user_id'].nunique()
num_distinct_users

464573

In [9]:
data_df = df_users_raw.groupby('track_id').size().reset_index(name='count')

In [10]:
data_df

,track_id,count
0,TRAAAED128E0783FAB,161
1,TRAAAHD128F42635A5,2
2,TRAAAQO12903CD8E1C,1
3,TRAABJS128F9325C99,945
4,TRAABXA128F9326351,18
...,...,...
30454,TRZZYMU128E0792400,1140
30455,TRZZZCL128F428BB80,19
30456,TRZZZCN128F9317A03,15
30457,TRZZZHL128F9329CFB,116


In [11]:
df_music.head()

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4


In [12]:
df_music_info = df_music[['track_id', 'name', 'artist', 'energy']]
df_music_info.head()

,track_id,name,artist,energy
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,0.918
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,0.892
2,TROUVHL128F426C441,Come as You Are,Nirvana,0.826
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,0.664
4,TRLNZBD128F935E4D8,Creep,Radiohead,0.430


In [13]:
df_music_count = df_music_info.merge(
    df_users.groupby('track_id').size().reset_index(name='playcount'),
    on='track_id',
    how='left'
)
df_music_count = df_music_count.fillna(0)
print(df_music_count.shape)
df_music_count.head()

(50683, 5)


,track_id,name,artist,energy,playcount
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,0.918,32.0
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,0.892,283.0
2,TROUVHL128F426C441,Come as You Are,Nirvana,0.826,0.0
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,0.664,17.0
4,TRLNZBD128F935E4D8,Creep,Radiohead,0.430,0.0


In [14]:
# df_music_count = df_music_info.merge(
#     df_users.groupby('track_id').size().reset_index(name='playcount'),
#     on='track_id',
#     how='inner'
# )
# print(df_music_count.shape)
# df_music_count.head()

## Interaction Matrix

In [15]:
df_users_agg = df_users.groupby('user_id')['playcount'].agg(
    #total_playcount='sum',
    max_playcount='max'
).reset_index()
df_users_agg = df_users_agg.rename(columns={'playcount': 'max_playcount'})

In [16]:
df_users_rating = df_users.merge(df_users_agg, on='user_id')

We create 2 interaction matrix
- Binary version: 1 indicates a user has interacted with the song and 0 indicates he has not
- Weighted version: Rating in range (0,1], calculated as (user playcount) / (playcount of user's most listened song). Gives more rating to those more listened by a user

In [17]:
df_users_rating['rating'] = df_users_rating['playcount'] / df_users_rating['max_playcount']
df_users_rating[:30]

,track_id,user_id,playcount,max_playcount,rating
0,TRIRLYL128F42539D1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50
1,TRFUPBA128F934F7E1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50
2,TRLQPQJ128F42AA94F,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50
3,TRTUCUY128F92E1D24,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50
4,TRHDDQG12903CB53EE,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50
5,TRGRDEC128F423C07D,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50
6,TRRYCBO128F932A2C7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50
7,TRUTULC128F4293712,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50
8,TRAAHSY128F147BB5C,b80344d063b5ccb3212f76538f3d9e43d87dca9e,2,2,1.00
9,TRDLMWP128F426BF6C,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50


In [18]:
df_users_agg[df_users_agg['user_id'] == 'b80344d063b5ccb3212f76538f3d9e43d87dca9e']

,user_id,max_playcount
333783,b80344d063b5ccb3212f76538f3d9e43d87dca9e,2


For the interaction matrix, we shall use a sparse matrix, due to users only listening to a small subset of the songs

In [19]:
from scipy.sparse import csr_matrix, csc_matrix

We cant reproduce next line because it needs 218 GB of RAM to create the table

In [20]:
#df_music_pivoted = df_users.pivot(index='user_id', columns='track_id', values='rating').fillna(0)

We codify each user and song to a unique integer ID

In [21]:
user_codes, user_uniques = pd.factorize(df_users['user_id'])
track_codes, track_uniques = pd.factorize(df_users['track_id'])

We create the interaction matrix as a sparse csr matrix for user-based collaborative filtering and as csc for item-based collaborative filtering

In [22]:
interaction_matrix_csr_binary = csr_matrix((np.ones(len(df_users)), (user_codes, track_codes)),shape=(len(user_uniques), len(track_uniques)))
interaction_matrix_csr_weighted = csr_matrix((df_users_rating['rating'], (user_codes, track_codes)),shape=(len(user_uniques), len(track_uniques)))

In [23]:
interaction_matrix_csc_binary = csc_matrix((np.ones(len(df_users)), (user_codes, track_codes)),shape=(len(user_uniques), len(track_uniques)))
interaction_matrix_csc_weighted = csc_matrix((df_users_rating['rating'], (user_codes, track_codes)),shape=(len(user_uniques), len(track_uniques)))

In [24]:
interaction_matrix_csr_binary.shape

(464573, 30201)

In [25]:
interaction_matrix_csc_weighted.shape

(464573, 30201)

In [26]:
# Calcular el espacio en memoria ocupado por interaction_matrix en MB
memory_usage_bytes = (
    interaction_matrix_csc_weighted.data.nbytes +
    interaction_matrix_csc_weighted.indptr.nbytes +
    interaction_matrix_csc_weighted.indices.nbytes
)
memory_usage_mb = memory_usage_bytes / (1024 ** 2)
print(f"Espacio en memoria de interaction_matrix: {memory_usage_mb:.2f} MB")

Espacio en memoria de interaction_matrix: 95.47 MB


The sparse matrix has a size of only 114.81 MB, while the normal matrix as a dataframe was 218 GB

In [27]:
interaction_matrix_csc_binary[:10, :30].todense()

matrix([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.

In [28]:
interaction_matrix_csc_weighted[:10, :30].todense()

matrix([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
         0.5       , 0.5       , 0.5       , 1.        , 0.5       ,
         0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
         0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.25      , 0.5       , 1.        , 0.25      , 0.25      ,
         0.5       , 0.75      , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ,

## Item based collaborative filtering

We use cosine similarity as a similarity metric. We can precalculate the similarities.

In [29]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csc_matrix

# Calculate item-based (track-based) cosine similarity
# Transpose interaction_matrix_weighted to get items as rows
item_based_similarity = cosine_similarity(interaction_matrix_csc_weighted.T, dense_output=False)
item_based_similarity = csc_matrix(item_based_similarity)

In [30]:
memory_usage_bytes = (
    item_based_similarity.data.nbytes +
    item_based_similarity.indptr.nbytes +
    item_based_similarity.indices.nbytes
)
memory_usage_mb = memory_usage_bytes / (1024 ** 2)
print(f"Espacio en memoria de item_based_similarity: {memory_usage_mb:.2f} MB")

Espacio en memoria de item_based_similarity: 934.39 MB


Only 7 seconds of execution

In [31]:
item_based_similarity.shape

(30201, 30201)

In [32]:
class ItemBasedRecommender:
    def __init__(self, interaction_matrix, item_similarity, track_uniques, df_music_info):
        self.interaction_matrix = interaction_matrix
        self.item_similarity = item_similarity
        self.track_uniques = track_uniques
        self.df_music_info = df_music_info
        self.user_index = None
        self.recommendations = None # List of tuples (track_id, energy, similarity, has been recommended)

    def make_recommendations(self, user_index, n=100):
        self.user_index = user_index
        user_ratings = self.interaction_matrix[self.user_index]
        interacted = user_ratings.nonzero()[1]
        similarities = self.item_similarity.dot(user_ratings.T).toarray().ravel()
        similarities[interacted] = -np.inf
        top_n_index = np.argpartition(similarities, -n)[-n:]
        top_n_index = top_n_index[np.argsort(similarities[top_n_index])[::-1]]

        # for index in top_n_index:
        #     print(f"Track ID: {self.track_uniques[index]}, Similarity: {similarities[index]}")

        track_ids = self.track_uniques[top_n_index].tolist()
        df_filtered = self.df_music_info.set_index('track_id').loc[track_ids][['energy']].reset_index()
        index_to_similarity = {idx: similarities[idx] for idx in top_n_index}

        self.recommendations = [(track_id, energy, index_to_similarity[self.track_uniques.tolist().index(track_id)], False) for track_id, energy in df_filtered.itertuples(index=False, name=None)]

    
    def recommend_song(self, energy, energy_margin=0.05):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        
        closest_track_index = None
        distance_to_energy = float('inf')

        for i, (track_id, track_energy, similarity, has_been_recommended) in enumerate(self.recommendations):
            distance = abs(track_energy - energy)

            if not has_been_recommended and distance <= energy_margin:
                self.recommendations[i] = (track_id, track_energy, similarity, True)
                return (track_id, track_energy)
            
            if not has_been_recommended and distance < distance_to_energy:
                closest_track_index = i
                distance_to_energy = distance
        
        if closest_track_index is not None:
            track_id, track_energy, _, _= self.recommendations[closest_track_index]
            self.recommendations[closest_track_index] = (track_id, track_energy, similarity, True)
            return (track_id, track_energy)

        raise ValueError("All recommendations have already been recommended")


    def get_recommendations(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return self.recommendations


    def get_recommendations_ids(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return [track_id for track_id, _, _, _ in self.recommendations]
    
    def get_recommendations_info(self):
        track_ids_ordered = [track_id for track_id, _, _, _ in self.recommendations]
        df_ordered = self.df_music_info.set_index('track_id').loc[track_ids_ordered].reset_index()
        return df_ordered

### Example

In [33]:
user_index = 0 #User for wich recommendations will be made

Songs the user has listened to

In [34]:
user_listened_songs = df_users[df_users['user_id'] == user_uniques[user_index]].track_id
df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

,track_id,name,artist,energy
79,TRAAHSY128F147BB5C,Speed of Sound,Coldplay,0.8980
796,TRDLMWP128F426BF6C,Ragged Wood,Fleet Foxes,0.6850
1158,TRPUGUW128F426BF6F,He Doesn't Know Why,Fleet Foxes,0.5580
1743,TRRYCBO128F932A2C7,Love Shack,The B-52's,0.9020
2350,TRFUPBA128F934F7E1,Paper Gangsta,Lady Gaga,0.5300
2585,TRVODUZ128F934D094,All That We Perceive,Thievery Corporation,0.8340
2722,TRHDDQG12903CB53EE,Heaven's Gonna Burn Your Eyes,Thievery Corporation,0.3370
5416,TRTUCUY128F92E1D24,Stacked Actors,Foo Fighters,0.9340
5428,TRLXSNR128F429361D,Generator,Foo Fighters,0.9240
5819,TRADVZX128F426BF79,Sun Giant,Fleet Foxes,0.0626


In [35]:
item_based_recommender = ItemBasedRecommender(interaction_matrix_csc_binary, item_based_similarity, track_uniques, df_music_info)
item_based_recommender.make_recommendations(user_index, n=20)
item_based_recommender.get_recommendations_info().head(50)

,track_id,name,artist,energy
0,TRDJRES128F933B4BA,Quiet Houses,Fleet Foxes,0.6680
1,TRKOHVA128F426BF70,Heard Them Stirring,Fleet Foxes,0.4700
2,TRQEMRN128F933B4B2,Sun It Rises,Fleet Foxes,0.2750
3,TRKABMK128F426BF72,Meadowlarks,Fleet Foxes,0.1660
4,TRGEIDA128F933B4B8,Tiger Mountain Peasant Song,Fleet Foxes,0.1460
5,TRVRIBZ128F426BF71,Your Protector,Fleet Foxes,0.5370
6,TRLXFKL128F426BF75,Oliver James,Fleet Foxes,0.0635
7,TRUJOHU128F424E6A6,White Winter Hymnal,Fleet Foxes,0.4970
8,TRSBMHN128F426BF7E,Innocent Son,Fleet Foxes,0.0201
9,TRVCFUI128F92E1D42,Live-In Skin,Foo Fighters,0.9670


In [36]:
recommended_song = item_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
8610,TRBIVWU128F92CA9D2,Skin and Bones,Foo Fighters,0.396


In [37]:
recommended_song = item_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
1272,TRKOHVA128F426BF70,Heard Them Stirring,Fleet Foxes,0.47


In [38]:
recommended_song = item_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
5152,TRUJOHU128F424E6A6,White Winter Hymnal,Fleet Foxes,0.497


### Experiments

#### Weighted vs binary matrix

We will compare the results with weighted and binary ratings

In [39]:
def compare_lists(list1, list2):
    min_len = min(len(list1), len(list2))

    for i in range(len(list1)):
        if list1[i] != list2[i]:
            print(f"They defer in position {i}")
            break
    else:
        print("They are the same for all positions up to the minimum length.")


#Random subset of users
random_generator = np.random.default_rng(seed=RANDOM_STATE)
random_users = random_generator.choice(np.arange(len(user_uniques)), size=20, replace=False)

first_different_recommendation = []

for rand_user in random_users:
    item_based_recommender = ItemBasedRecommender(interaction_matrix_csc_weighted, item_based_similarity, track_uniques, df_music_info)
    item_based_recommender.make_recommendations(rand_user, n=200)
    weighted_recommendations = item_based_recommender.get_recommendations_ids()

    item_based_recommender = ItemBasedRecommender(interaction_matrix_csc_binary, item_based_similarity, track_uniques, df_music_info)
    item_based_recommender.make_recommendations(rand_user, n=200)
    binary_recommendations = item_based_recommender.get_recommendations_ids()

    for i in range(len(weighted_recommendations)):
        if weighted_recommendations[i] != binary_recommendations[i]:
            first_different_recommendation.append(i)
            break
    else:
        first_different_recommendation.append(-1)

print("First different recommendation for each user: (-1 = every recommendation is the same)")
print(first_different_recommendation)

First different recommendation for each user: (-1 = every recommendation is the same)
[0, 0, 0, 0, 2, 8, -1, 0, 3, 11, 0, 0, 0, 0, 0, 1, 2, 2, 3, 8]


We can see that recommendations defer in most of the cases. We will analyse a case where the recommendations are not the same

In [40]:
analized_user = random_users[0]

In [41]:
item_based_recommender = ItemBasedRecommender(interaction_matrix_csc_weighted, item_based_similarity, track_uniques, df_music_info)
item_based_recommender.make_recommendations(analized_user, n=200)
weighted_recommendations = item_based_recommender.get_recommendations_info()

item_based_recommender = ItemBasedRecommender(interaction_matrix_csc_binary, item_based_similarity, track_uniques, df_music_info)
item_based_recommender.make_recommendations(analized_user, n=200)
binary_recommendations = item_based_recommender.get_recommendations_info()

In [42]:
weighted_recommendations.head(20)

,track_id,name,artist,energy
0,TRDGUPO128F9332CF6,Down To Earth,Bungle,0.904
1,TRVSJOM12903CD2DC1,One Less Lonely Girl,Justin Bieber,0.750
2,TRYEGSH12903CD2DCE,Overboard,Justin Bieber,0.790
3,TRZEDRT12903CD2DCC,Runaway Love,Justin Bieber,0.633
4,TRCKWGF12903CD2DCD,Never Let You Go,Third Eye Blind,0.940
5,TRTKLFX12903CD2DC2,First Dance,Justin Bieber,0.520
6,TRZYAGJ128F9332CEF,Favorite Girl,Justin Bieber,0.681
7,TRLVQME128F931BAF3,Vanilla Twilight,Owl City,0.523
8,TRIQIAF128E0788692,Try a Little Tenderness,Michael Bublé,0.245
9,TRCPXID128F92D5D3C,Halo,Depeche Mode,0.508


In [43]:
binary_recommendations.head(20)

,track_id,name,artist,energy
0,TRVSJOM12903CD2DC1,One Less Lonely Girl,Justin Bieber,0.750
1,TRDGUPO128F9332CF6,Down To Earth,Bungle,0.904
2,TRYEGSH12903CD2DCE,Overboard,Justin Bieber,0.790
3,TRZEDRT12903CD2DCC,Runaway Love,Justin Bieber,0.633
4,TRCKWGF12903CD2DCD,Never Let You Go,Third Eye Blind,0.940
5,TRIQIAF128E0788692,Try a Little Tenderness,Michael Bublé,0.245
6,TRLVQME128F931BAF3,Vanilla Twilight,Owl City,0.523
7,TRTKLFX12903CD2DC2,First Dance,Justin Bieber,0.520
8,TRZYAGJ128F9332CEF,Favorite Girl,Justin Bieber,0.681
9,TRLNFKN128F931BAF2,The Tip Of The Iceberg,Owl City,0.772


We can see that even the order is not the same, most of the items are. Now we will calculate how many of the items recomended with the weighted ratings are recommended using the binary ratings

In [44]:
weighted_ids = set(weighted_recommendations['track_id'].head(50))
binary_ids = set(binary_recommendations['track_id'].head(50))
porcentaje = len(weighted_ids & binary_ids) / len(weighted_ids) * 100
print(f"Percentage of wieghted recommendations that are in binary recomendations: {porcentaje:.2f}%")

Percentage of wieghted recommendations that are in binary recomendations: 96.00%


96% of the items are the same in the first 50 recommendations of both systems. So, for this user, the use of weighted or binary rating system does not seem to have much impact. We will study the overlap of the other users of the subset

In [45]:
percentage_overlap_20 = []
percentage_overlap_50 = []
percentage_overlap_200 = []

for rand_user in random_users:
    item_based_recommender = ItemBasedRecommender(interaction_matrix_csc_weighted, item_based_similarity, track_uniques, df_music_info)
    item_based_recommender.make_recommendations(rand_user, n=200)
    weighted_recommendations = item_based_recommender.get_recommendations_info()

    item_based_recommender = ItemBasedRecommender(interaction_matrix_csc_binary, item_based_similarity, track_uniques, df_music_info)
    item_based_recommender.make_recommendations(rand_user, n=200)
    binary_recommendations = item_based_recommender.get_recommendations_info()

    weighted_ids_20 = set(weighted_recommendations['track_id'].head(20))
    binary_ids_20 = set(binary_recommendations['track_id'].head(20))

    weighted_ids_50 = set(weighted_recommendations['track_id'].head(50))
    binary_ids_50 = set(binary_recommendations['track_id'].head(50))

    weighted_ids_200 = set(weighted_recommendations['track_id'])
    binary_ids_200 = set(binary_recommendations['track_id'])


    overlap = round(len(weighted_ids_20 & binary_ids_20) / len(weighted_ids_20) * 100, 2)
    percentage_overlap_20.append(overlap)

    overlap = round(len(weighted_ids_50 & binary_ids_50) / len(weighted_ids_50) * 100, 2)
    percentage_overlap_50.append(overlap)

    overlap = round(len(weighted_ids_200 & binary_ids_200) / len(weighted_ids_200) * 100, 2)
    percentage_overlap_200.append(overlap)

print("Percentage of overlap between weighted and binary recommendations for each user:")
print(f"First 20 recommendations: {percentage_overlap_20}")
print(f"First 50 recommendations: {percentage_overlap_50}")
print(f"First 200 recommendations: {percentage_overlap_200}")

print(f"Average percentage of overlap for 20 first recommendations: {np.mean(percentage_overlap_20):.2f}%")
print(f"Median percentage of overlap for 20 first recommendations: {np.median(percentage_overlap_20):.2f}%")

print(f"Average percentage of overlap for 50 first recommendations: {np.mean(percentage_overlap_50):.2f}%")
print(f"Median percentage of overlap for 50 first recommendations: {np.median(percentage_overlap_50):.2f}%")

print(f"Average percentage of overlap for 200 first recommendations: {np.mean(percentage_overlap_200):.2f}%")
print(f"Median percentage of overlap for 200 first recommendations: {np.median(percentage_overlap_200):.2f}%")

Percentage of overlap between weighted and binary recommendations for each user:
First 20 recommendations: [95.0, 50.0, 70.0, 25.0, 65.0, 100.0, 100.0, 40.0, 60.0, 95.0, 85.0, 60.0, 65.0, 45.0, 40.0, 70.0, 95.0, 75.0, 95.0, 100.0]
First 50 recommendations: [96.0, 58.0, 72.0, 72.0, 94.0, 88.0, 100.0, 66.0, 74.0, 100.0, 58.0, 82.0, 80.0, 86.0, 72.0, 74.0, 78.0, 96.0, 86.0, 86.0]
First 200 recommendations: [95.5, 69.0, 80.0, 78.5, 89.0, 90.5, 100.0, 73.5, 93.0, 97.5, 85.5, 72.5, 78.0, 88.0, 55.5, 80.5, 75.0, 95.0, 80.5, 87.5]
Average percentage of overlap for 20 first recommendations: 71.50%
Median percentage of overlap for 20 first recommendations: 70.00%
Average percentage of overlap for 50 first recommendations: 80.90%
Median percentage of overlap for 50 first recommendations: 81.00%
Average percentage of overlap for 200 first recommendations: 83.22%
Median percentage of overlap for 200 first recommendations: 83.00%


We can see that there are not much differences in most of the cases using the weighted or binary rating method

## User based collaborative filtering

In [46]:
#user_similarity = cosine_similarity(interaction_matrix_csr_weighted, dense_output=False)

We cant perform that operation because the amount of RAM needed, so we can not precalculate the similarities

In [47]:
from sklearn.neighbors import NearestNeighbors

In [48]:
class UserBasedRecommender:
    def __init__(self, interaction_matrix, track_uniques, df_music_info, num_neighbors=10):
        self.interaction_matrix = interaction_matrix
        self.knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=num_neighbors + 1, n_jobs=-1) # num_neighbors+1 because the user itself is included in the neighbors and we will ignore it
        self.knn_model.fit(interaction_matrix)
        self.track_uniques = track_uniques
        self.df_music_info = df_music_info
        self.user_index = None
        self.recommendations = None # List of tuples (track_id, energy, has been recommended)

    def make_recommendations(self, user_index, n=5):
        self.user_index = user_index

        _, neighbors_indices = self.knn_model.kneighbors(self.interaction_matrix[user_index])

        neighbors_indices = neighbors_indices[:, 1:]  # Exclude the first index which is the user itself

        neighbors_interaction_matrix = self.interaction_matrix[neighbors_indices.flatten()]

        neighbors_mean_ratings = neighbors_interaction_matrix.mean(axis=0).A1 # Converts to 1D array

        user_ratings = self.interaction_matrix[user_index]
        interacted = user_ratings.nonzero()[1]
        neighbors_mean_ratings[interacted] = -np.inf #Asign -np.inf to already listened songs by the user

        top_n_index = np.argpartition(neighbors_mean_ratings, -n)[-n:]
        top_n_index = top_n_index[np.argsort(neighbors_mean_ratings[top_n_index])[::-1]]

        # for index in top_n_index:
        #     print(f"Track ID: {self.track_uniques[index]}, Mean Rating: {neighbors_mean_ratings[index]}")
        
        track_ids = self.track_uniques[top_n_index].tolist()

        df_filtered = self.df_music_info.set_index('track_id').loc[track_ids][['energy']].reset_index()

        # print(df_filtered)

        self.recommendations = [(track_id, energy, False) for track_id, energy in df_filtered.itertuples(index=False, name=None)]

    
    def recommend_song(self, energy, energy_margin=0.05):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        
        closest_track_index = None
        distance_to_energy = float('inf')

        for i, (track_id, track_energy, has_been_recommended) in enumerate(self.recommendations):
            distance = abs(track_energy - energy)

            if not has_been_recommended and distance <= energy_margin:
                self.recommendations[i] = (track_id, track_energy, True)
                return (track_id, track_energy)
            
            if not has_been_recommended and distance < distance_to_energy:
                closest_track_index = i
                distance_to_energy = distance
        
        if closest_track_index is not None:
            track_id, track_energy, _ = self.recommendations[closest_track_index]
            self.recommendations[closest_track_index] = (track_id, track_energy, True)
            return (track_id, track_energy)

        raise ValueError("All recommendations have already been recommended")


    def get_recommendations(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return self.recommendations


    def get_recommendations_ids(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return [track_id for track_id, _, _ in self.recommendations]
    
    
    def get_recommendations_info(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        track_ids_ordered = [track_id for track_id, _, _ in self.recommendations]
        df_ordered = self.df_music_info.set_index('track_id').loc[track_ids_ordered].reset_index()
        return df_ordered

### Example

In [49]:
user_index = 0 #User for wich recommendations will be made

In [50]:
user_listened_songs = df_users[df_users['user_id'] == user_uniques[user_index]].track_id
df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

,track_id,name,artist,energy
79,TRAAHSY128F147BB5C,Speed of Sound,Coldplay,0.8980
796,TRDLMWP128F426BF6C,Ragged Wood,Fleet Foxes,0.6850
1158,TRPUGUW128F426BF6F,He Doesn't Know Why,Fleet Foxes,0.5580
1743,TRRYCBO128F932A2C7,Love Shack,The B-52's,0.9020
2350,TRFUPBA128F934F7E1,Paper Gangsta,Lady Gaga,0.5300
2585,TRVODUZ128F934D094,All That We Perceive,Thievery Corporation,0.8340
2722,TRHDDQG12903CB53EE,Heaven's Gonna Burn Your Eyes,Thievery Corporation,0.3370
5416,TRTUCUY128F92E1D24,Stacked Actors,Foo Fighters,0.9340
5428,TRLXSNR128F429361D,Generator,Foo Fighters,0.9240
5819,TRADVZX128F426BF79,Sun Giant,Fleet Foxes,0.0626


In [51]:
user_based_recommender = UserBasedRecommender(interaction_matrix_csr_binary, track_uniques, df_music_info, num_neighbors=10)
user_based_recommender.make_recommendations(user_index, n=20)

In [52]:
# user_listened_songs = df_users[df_users['user_id'] == user_uniques[201670]].track_id
# df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

In [53]:
user_based_recommender.get_recommendations_info().head(20)

,track_id,name,artist,energy
0,TRVCFUI128F92E1D42,Live-In Skin,Foo Fighters,0.967
1,TRPGPDK12903CCC651,Bring Me To Life,Katherine Jenkins,0.563
2,TRJVHHB128F423786F,Aluga-se-vende,Móveis Coloniais de Acaju,0.799
3,TRYFJBZ128E0783C26,Two Way Monologue,Sondre Lerche,0.735
4,TRYHRDD128E0781847,Counter Spark,Sondre Lerche,0.648
5,TRFVBZA128F423787B,Do mesmo ar,Móveis Coloniais de Acaju,0.802
6,TRXWLLE128F92FC125,Hope You're Feeling Better,Santana,0.662
7,TRUKVPU128F147B59B,Love Vibration,Josh Rouse,0.663
8,TRBQCKI128F92E2DE5,Graveyard Disciples,Black Label Society,0.796
9,TRMNLOT128F1473B10,I Ain't Marching Anymore,Phil Ochs,0.492


In [54]:
recommended_song = user_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
34798,TRDQHPF12903CDA54E,Open All Night,Bruce Springsteen,0.386


In [55]:
recommended_song = user_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
30602,TRMKOST128E079939B,Immigrant Workforce,Bedouin Soundclash,0.47


In [56]:
recommended_song = user_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
32790,TRMNLOT128F1473B10,I Ain't Marching Anymore,Phil Ochs,0.492


### Experiments

#### Number of neighbors (k)

If we take the number of total users as neighbor count the system will recommend the songs that the user has not listened to in order of total play counts of all the users

In [70]:
user_index = 10 #User for wich recommendations will be made
user_listened_songs = df_users[df_users['user_id'] == user_uniques[user_index]].track_id
df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

,track_id,name,artist,energy
1249,TRZPVLJ128F148D2F7,Rock And Roll All Nite,Kiss,0.857
6047,TRAAKDG128F42A0ECB,Hips Don't Lie,Shakira,0.820
10474,TRIAYBJ128E0799A5B,It's Raining Again,Supertramp,0.715
10631,TROVGLQ128F4262EBD,I Hate Myself for Loving You,Joan Jett and the Blackhearts,0.946
19508,TRDGFON128F148D2F8,Cold Gin,Kiss,0.961
43748,TRVAXHF128F931FB04,Until You're Mine,Demi Lovato,0.873
43798,TRKJYHD128F931FB01,Gonna Get Caught,Demi Lovato,0.910


In [71]:
user_based_recommender = UserBasedRecommender(interaction_matrix_csr_binary, track_uniques, df_music_info, num_neighbors=num_distinct_users - 1)
user_based_recommender.make_recommendations(user_index, n=50)

In [72]:
user_based_recommender.get_recommendations_info().head(5)

,track_id,name,artist,energy
0,TRONYHY128F92C9D11,Revelry,Kings of Leon,0.748
1,TRUFTBY128F93450B8,Alejandro,Lady Gaga,0.800
2,TRPFYYL128F92F7144,Float On,Modest Mouse,0.893
3,TRXWAZC128F9314B3E,Heartbreak Warfare,John Mayer,0.572
4,TROMKCG128F9320C09,Uprising,Sabaton,0.866


In [73]:
num_distinct_users = df_users['user_id'].nunique()
track_playcounts = df_users.groupby('track_id')['user_id'].nunique().sort_values(ascending=False)
top_5_track_ids = track_playcounts.index[:5]
top_5_df = track_playcounts.loc[top_5_track_ids].reset_index()
top_5_df.columns = ['track_id', 'total_playcount']
top_5_df.merge(df_music_info, on='track_id')

,track_id,total_playcount,name,artist,energy
0,TRONYHY128F92C9D11,44469,Revelry,Kings of Leon,0.748
1,TRUFTBY128F93450B8,27497,Alejandro,Lady Gaga,0.800
2,TRPFYYL128F92F7144,23716,Float On,Modest Mouse,0.893
3,TRXWAZC128F9314B3E,23273,Heartbreak Warfare,John Mayer,0.572
4,TROMKCG128F9320C09,22088,Uprising,Sabaton,0.866


The top 5 recommendations are the same than the top 5 songs with more users who has listened them

#### Weighted vs binary matrix

In [ ]:
random_generator = np.random.default_rng(seed=RANDOM_STATE)
random_users = random_generator.choice(np.arange(len(user_uniques)), size=20, replace=False)

first_different_recommendation = []

for rand_user in random_users:
    user_based_recommender = UserBasedRecommender(interaction_matrix_csr_weighted, track_uniques, df_music_info, num_neighbors=10)
    user_based_recommender.make_recommendations(rand_user, n=200)
    weighted_recommendations = user_based_recommender.get_recommendations_ids()

    user_based_recommender = UserBasedRecommender(interaction_matrix_csr_binary, track_uniques, df_music_info, num_neighbors=10)
    user_based_recommender.make_recommendations(rand_user, n=200)
    binary_recommendations = user_based_recommender.get_recommendations_ids()

    for i in range(len(weighted_recommendations)):
        if weighted_recommendations[i] != binary_recommendations[i]:
            first_different_recommendation.append(i)
            break
    else:
        first_different_recommendation.append(-1)

print("First different recommendation for each user: (-1 = every recommendation is the same)")
print(first_different_recommendation)

First different recommendation for each user: (-1 = every recommendation is the same)
[0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]


Recommendations usually defer before than with item-based collaborative filtring

In [ ]:
analized_user = random_users[0]

user_based_recommender = UserBasedRecommender(interaction_matrix_csr_weighted, track_uniques, df_music_info, num_neighbors=10)
user_based_recommender.make_recommendations(analized_user, n=200)
weighted_recommendations = user_based_recommender.get_recommendations_info()

user_based_recommender = UserBasedRecommender(interaction_matrix_csr_binary, track_uniques, df_music_info, num_neighbors=10)
user_based_recommender.make_recommendations(analized_user, n=200)
binary_recommendations = user_based_recommender.get_recommendations_info()

In [ ]:
weighted_recommendations.head(20)

,track_id,name,artist,energy
0,TRYEGSH12903CD2DCE,Overboard,Justin Bieber,0.790
1,TRZYAGJ128F9332CEF,Favorite Girl,Justin Bieber,0.681
2,TRCKWGF12903CD2DCD,Never Let You Go,Third Eye Blind,0.940
3,TRDGUPO128F9332CF6,Down To Earth,Bungle,0.904
4,TRTKLFX12903CD2DC2,First Dance,Justin Bieber,0.520
5,TRVSJOM12903CD2DC1,One Less Lonely Girl,Justin Bieber,0.750
6,TRIIXNL128F425EAA1,Sweetest Girl (Dollar Bill),Wyclef Jean,0.746
7,TRPTVTR128F93450C6,"Beautiful, Dirty, Rich",Lady Gaga,0.903
8,TRUQAQG128F42748B5,Make Yourself,Incubus,0.979
9,TRZEDRT12903CD2DCC,Runaway Love,Justin Bieber,0.633


In [ ]:
binary_recommendations.head(20)

,track_id,name,artist,energy
0,TRIQIAF128E0788692,Try a Little Tenderness,Michael Bublé,0.245
1,TRCPXID128F92D5D3C,Halo,Depeche Mode,0.508
2,TRUFTBY128F93450B8,Alejandro,Lady Gaga,0.800
3,TROMKCG128F9320C09,Uprising,Sabaton,0.866
4,TRVSJOM12903CD2DC1,One Less Lonely Girl,Justin Bieber,0.750
5,TRWAQOC12903CB84CA,Sexy Bitch,David Guetta,0.595
6,TRAALAH128E078234A,Bitter Sweet Symphony,The Verve,0.907
7,TRBQIET128F9310B55,Razor Boy,Steely Dan,0.447
8,TRROQXP128F9309C07,Dragonheads,Ensiferum,0.960
9,TRVTFPZ128F42AD91D,Black Heart Romance,My Dying Bride,0.635


In [ ]:
weighted_ids = set(weighted_recommendations['track_id'].head(50))
binary_ids = set(binary_recommendations['track_id'].head(50))
porcentaje = len(weighted_ids & binary_ids) / len(weighted_ids) * 100
print(f"Percentage of wieghted recommendations that are in binary recomendations: {porcentaje:.2f}%")

Percentage of wieghted recommendations that are in binary recomendations: 38.00%


Only 38% of the items are the same in the first 50 recommendations. We will study this overlap for the full subset of users

In [ ]:
percentage_overlap_20 = []
percentage_overlap_50 = []
percentage_overlap_200 = []

for rand_user in random_users:
    user_based_recommender = UserBasedRecommender(interaction_matrix_csr_weighted, track_uniques, df_music_info, num_neighbors=10)
    user_based_recommender.make_recommendations(rand_user, n=200)
    weighted_recommendations = user_based_recommender.get_recommendations_info()

    user_based_recommender = UserBasedRecommender(interaction_matrix_csr_binary, track_uniques, df_music_info, num_neighbors=10)
    user_based_recommender.make_recommendations(rand_user, n=200)
    binary_recommendations = user_based_recommender.get_recommendations_info()

    weighted_ids_20 = set(weighted_recommendations['track_id'].head(20))
    binary_ids_20 = set(binary_recommendations['track_id'].head(20))

    weighted_ids_50 = set(weighted_recommendations['track_id'].head(50))
    binary_ids_50 = set(binary_recommendations['track_id'].head(50))

    weighted_ids_200 = set(weighted_recommendations['track_id'])
    binary_ids_200 = set(binary_recommendations['track_id'])


    overlap = round(len(weighted_ids_20 & binary_ids_20) / len(weighted_ids_20) * 100, 2)
    percentage_overlap_20.append(overlap)

    overlap = round(len(weighted_ids_50 & binary_ids_50) / len(weighted_ids_50) * 100, 2)
    percentage_overlap_50.append(overlap)

    overlap = round(len(weighted_ids_200 & binary_ids_200) / len(weighted_ids_200) * 100, 2)
    percentage_overlap_200.append(overlap)

print("Percentage of overlap between weighted and binary recommendations for each user:")
print(f"First 20 recommendations: {percentage_overlap_20}")
print(f"First 50 recommendations: {percentage_overlap_50}")
print(f"First 200 recommendations: {percentage_overlap_200}")

print(f"Average percentage of overlap for 20 first recommendations: {np.mean(percentage_overlap_20):.2f}%")
print(f"Median percentage of overlap for 20 first recommendations: {np.median(percentage_overlap_20):.2f}%")

print(f"Average percentage of overlap for 50 first recommendations: {np.mean(percentage_overlap_50):.2f}%")
print(f"Median percentage of overlap for 50 first recommendations: {np.median(percentage_overlap_50):.2f}%")

print(f"Average percentage of overlap for 200 first recommendations: {np.mean(percentage_overlap_200):.2f}%")
print(f"Median percentage of overlap for 200 first recommendations: {np.median(percentage_overlap_200):.2f}%")

Percentage of overlap between weighted and binary recommendations for each user:
First 20 recommendations: [15.0, 5.0, 70.0, 5.0, 5.0, 10.0, 30.0, 0.0, 15.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 15.0, 0.0, 15.0, 20.0, 25.0]
First 50 recommendations: [38.0, 2.0, 64.0, 6.0, 2.0, 6.0, 30.0, 6.0, 12.0, 6.0, 14.0, 4.0, 4.0, 6.0, 0.0, 10.0, 8.0, 12.0, 34.0, 68.0]
First 200 recommendations: [84.5, 31.5, 91.0, 62.5, 74.5, 36.0, 64.5, 66.0, 72.0, 72.0, 63.0, 67.5, 70.0, 57.0, 64.0, 74.0, 37.5, 63.0, 83.5, 91.5]
Average percentage of overlap for 20 first recommendations: 12.50%
Median percentage of overlap for 20 first recommendations: 10.00%
Average percentage of overlap for 50 first recommendations: 16.60%
Median percentage of overlap for 50 first recommendations: 7.00%
Average percentage of overlap for 200 first recommendations: 66.28%
Median percentage of overlap for 200 first recommendations: 66.75%


We can see percentajes are very small and much smaller than in item based recommendation. In this case, the type of matrix used is very important. We will compare the k neighbours selected with each matrix. We start with a low overlap percentage user.

Do not reproduce next cell, it depends on a comented print

In [ ]:
analized_user = random_users[3]

user_based_recommender = UserBasedRecommender(interaction_matrix_csr_weighted, track_uniques, df_music_info, num_neighbors=10)
user_based_recommender.make_recommendations(analized_user, n=200)

user_based_recommender = UserBasedRecommender(interaction_matrix_csr_binary, track_uniques, df_music_info, num_neighbors=10)
user_based_recommender.make_recommendations(analized_user, n=200)

[[282688  77172 198958 427635 153900 350047 246860 130032 251243  82370]]
[[ 55830 352595 203280  98078 456508 266031 428437 135575  54447 282915]]


In [ ]:
weighted_k_neighbors = [282688   , 77172  , 163871, 12384, 399104, 388294, 89958, 299968, 416558, 382220]
binary_k_neighbors = [55830  , 352595  , 139912, 397766, 263995, 85450, 23316, 456655, 267416, 190488]
most_similar_weighted = weighted_k_neighbors[0]
most_similar_binary = binary_k_neighbors[0]

We can see the neighbours are all diferent. We will compare them with the original user

Original user:

In [ ]:
user_listened_songs = df_users[df_users['user_id'] == user_uniques[analized_user]].track_id
df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

,track_id,name,artist,energy
17193,TRXRFBY128F93234B5,Hoedown Throwdown,Miley Cyrus,0.801
19907,TRRVEDO128F147B0C9,These Boots,Megadeth,0.996
23220,TRGCHLH12903CB7352,Party In The U.S.A.,The Barden Bellas,0.360
33021,TRKCQHG128F92E0C1C,Good Directions,Billy Currington,0.724
33054,TRQTFUV128F426BFC7,Must Be Doin' Somethin' Right,Billy Currington,0.615
33071,TRHJSEZ128F1459DCC,I Got A Feelin',Billy Currington,0.727
33114,TRMLNIE128F429FEFA,You Make Me Want To,Luke Bryan,0.695
33266,TRUWKDV128F92DE274,A Perfectly Good Heart,Taylor Swift,0.751
33574,TRKOZVY12903CFDBD0,Hell On The Heart,Eric Church,0.898
43575,TRGBOPI128F14955AD,Everyone Like Me,Thousand Foot Krutch,0.922


Most similar user obtained with a weighted interaction matrix

In [ ]:
user_listened_songs = df_users[df_users['user_id'] == user_uniques[most_similar_weighted]].track_id
df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

,track_id,name,artist,energy
2300,TRSUSWW128F93463BF,Imma Be,Black Eyed Peas,0.539
6062,TRRXGAK128EF349F1A,The Way I Are,Timbaland,0.946
14481,TRSIMWV12903CD1951,Drop It Low,Ester Dean,0.714
19998,TRTUNTL128F429B125,Daggers Speak Louder Than Words,Alesana,0.950
20005,TRRQMQZ128F429B126,The Last Three Letters,Alesana,0.923
31537,TRYFDNR128F4260C5E,No Air duet with Chris Brown,Jordin Sparks,0.750
33266,TRUWKDV128F92DE274,A Perfectly Good Heart,Taylor Swift,0.751
34965,TRNGAJS128F9305B10,"Stay Close, Don't Go",Secondhand Serenade,0.829
35128,TRLOHOT128F42631FA,Endings Without Stories,Alesana,0.953
41358,TRYOLWR128F92EDA98,Early Mourning,Alesana,0.245


Only 1 songs is the same

In [ ]:
user_listened_songs = df_users[df_users['user_id'] == user_uniques[most_similar_binary]].track_id
df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

,track_id,name,artist,energy
19907,TRRVEDO128F147B0C9,These Boots,Megadeth,0.996
33021,TRKCQHG128F92E0C1C,Good Directions,Billy Currington,0.724
33071,TRHJSEZ128F1459DCC,I Got A Feelin',Billy Currington,0.727
33091,TRVABNY128F147B0C7,Sinners Like Me,Eric Church,0.538
33574,TRKOZVY12903CFDBD0,Hell On The Heart,Eric Church,0.898
33729,TRDBIRM128F147B0D1,Livin' Part Of Life,Eric Church,0.648


4 songs are the same

With the second most similar user:

In [ ]:
user_listened_songs = df_users[df_users['user_id'] == user_uniques[weighted_k_neighbors[1]]].track_id
df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

,track_id,name,artist,energy
45,TRQFICK128F92F5190,Toxicity,System of a Down,0.873
142,TRYALRP128F92F521E,Aerials,System of a Down,0.924
4708,TREXDDS128F4277776,One Man Drinking Games,Mayday Parade,0.860
19983,TRYPIMO128F422BFCE,Two As One,From First to Last,0.995
20046,TRIRJOU128F145F994,Close Yet Far,CKY,0.796
20202,TRCNVNY128F932DFB5,Feed Me To The Forest,The Blood Brothers,0.923
26769,TRNLALU12903CF43BC,Shoeshine For Neptune,Arsonists Get All The Girls,0.981
33266,TRUWKDV128F92DE274,A Perfectly Good Heart,Taylor Swift,0.751
35359,TRXBJZT128F42638EE,Whatever I Say is Royal Ocean,Dance Gavin Dance,0.617
42423,TRVSWMJ128F4254C8D,Tangled Up In Me,Skye Sweetnam,0.719


1 is the same

In [ ]:
user_listened_songs = df_users[df_users['user_id'] == user_uniques[binary_k_neighbors[1]]].track_id
df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

,track_id,name,artist,energy
1136,TRUFOZX128F92FDB8B,The New Year,Death Cab for Cutie,0.761
2700,TRNFVQI128F931BAEA,The Saltwater Room,Owl City,0.686
6074,TRWBTFY128F1484B23,Fergalicious,Fergie,0.585
8873,TRGENMC128F4263658,How Sweet It Is,Michael Bublé,0.624
17141,TRQOJUO12903CA280A,Why Don't We Just Dance,Josh Turner,0.799
17193,TRXRFBY128F93234B5,Hoedown Throwdown,Miley Cyrus,0.801
23220,TRGCHLH12903CB7352,Party In The U.S.A.,The Barden Bellas,0.360
33020,TRQYEBP128F92E4F05,People Are Crazy,Billy Currington,0.714
33021,TRKCQHG128F92E0C1C,Good Directions,Billy Currington,0.724
33054,TRQTFUV128F426BFC7,Must Be Doin' Somethin' Right,Billy Currington,0.615


6 are the same

With the binary option, the system identifies better people who has listened the same songs, but it does not distinguish whether the songs have a high or low playcount

### No KNN

In [ ]:
class NoKNNUserBasedRecommender:
    def __init__(self, interaction_matrix, track_uniques, df_music_info, num_neighbors=10):
        self.interaction_matrix = interaction_matrix
        self.track_uniques = track_uniques
        self.df_music_info = df_music_info
        self.num_neighbors = num_neighbors
        self.user_index = None
        self.recommendations = None # List of tuples (track_id, energy, has been recommended)

    def make_recommendations(self, user_index, n=5):
        self.user_index = user_index

        user_ratings = self.interaction_matrix[user_index]

        user_similarities = cosine_similarity(user_ratings, self.interaction_matrix).flatten()

        user_similarities[user_index] = -np.inf

        top_n_neighbors_index = np.argpartition(user_similarities, -self.num_neighbors)[-self.num_neighbors:]
        top_n_neighbors_index = top_n_neighbors_index[np.argsort(user_similarities[top_n_neighbors_index])[::-1]]

        #print(top_n_neighbors_index)

        neighbors_mean_ratings = self.interaction_matrix[top_n_neighbors_index].mean(axis=0).A1

        user_ratings = self.interaction_matrix[user_index]
        interacted = user_ratings.nonzero()[1]
        neighbors_mean_ratings[interacted] = -np.inf #Asign -np.inf to already listened songs by the user

        top_n_index = np.argpartition(neighbors_mean_ratings, -n)[-n:]
        top_n_index = top_n_index[np.argsort(neighbors_mean_ratings[top_n_index])[::-1]]

        rating_list = []
        for index in top_n_index:
            rating_list.append(float(neighbors_mean_ratings[index]))

        rating_array = np.array(rating_list)
        
        # print('Num of elements of rating_list > 0:', np.sum(rating_array > 0))
        # print('Num of elements of rating_list = 0:', np.sum(rating_array == 0))

        # for index in top_n_index:
        #     print(f"Track ID: {self.track_uniques[index]}, Mean Rating: {neighbors_mean_ratings[index]}")
        
        track_ids = self.track_uniques[top_n_index].tolist()

        df_filtered = self.df_music_info.set_index('track_id').loc[track_ids][['energy']].reset_index()


        self.recommendations = [(track_id, energy, False) for track_id, energy in df_filtered.itertuples(index=False, name=None)]

    
    def recommend_song(self, energy, energy_margin=0.05):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        
        closest_track_index = None
        distance_to_energy = float('inf')

        for i, (track_id, track_energy, has_been_recommended) in enumerate(self.recommendations):
            distance = abs(track_energy - energy)

            if not has_been_recommended and distance <= energy_margin:
                self.recommendations[i] = (track_id, track_energy, True)
                return (track_id, track_energy)
            
            if not has_been_recommended and distance < distance_to_energy:
                closest_track_index = i
                distance_to_energy = distance
        
        if closest_track_index is not None:
            track_id, track_energy, _ = self.recommendations[closest_track_index]
            self.recommendations[closest_track_index] = (track_id, track_energy, True)
            return (track_id, track_energy)

        raise ValueError("All recommendations have already been recommended")


    def get_recommendations(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return self.recommendations


    def get_recommendations_ids(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return [track_id for track_id, _, _ in self.recommendations]
    
    
    def get_recommendations_info(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        track_ids_ordered = [track_id for track_id, _, _ in self.recommendations]
        df_ordered = self.df_music_info.set_index('track_id').loc[track_ids_ordered].reset_index()
        return df_ordered

## Matrix Factorization: Alternating Least Squares (ALS)

Aleternating Least Squares performs very well for sparse and implicit data. So we shall calculate the interaction matrix as the playcount or as proposed in the original ALS paper:

$$
\text{confidence}_{ui} = 1 + \alpha \cdot r_{ui}
$$

We also calculate it with a bm25 weighting scheme to reduce the effect of a single user listening too many times to the same song and to direct somo of the confidence score to the most popular items, as proposed on the Implicit library documentation


In [96]:
ALPHA = 40 #Value proposed in the paper "A Matrix Factorization Approach to Online Recommendation with Implicit Feedback" by Hu et al. (2008)

confidence_values = 1 + ALPHA * df_users_rating['playcount']

interaction_matrix_user_item_paper = csr_matrix(
    (confidence_values, (user_codes, track_codes)),
    shape=(len(user_uniques), len(track_uniques))
)


In [100]:
from implicit.nearest_neighbours import bm25_weight


interaction_matrix_user_item_bm25 = csr_matrix(
    (df_users_rating['playcount'], (user_codes, track_codes)),
    shape=(len(user_uniques), len(track_uniques))
)

interaction_matrix_user_item = bm25_weight(interaction_matrix_user_item_bm25).tocsr() #We use K1 and B parameters as default values (K1=100, B=0.8)


In [101]:
from implicit.als import AlternatingLeastSquares

In [132]:
class ALSRecommender:
    def __init__(self, interaction_matrix, track_uniques, df_music_info, als_model=None):
        self.interaction_matrix = interaction_matrix
        self.track_uniques = track_uniques
        self.df_music_info = df_music_info

        if als_model is None:
            self.als_model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20, num_threads=-1, random_state=RANDOM_STATE)
            self.als_model.fit(self.interaction_matrix)
        else:
            self.als_model = als_model

        self.user_index = None
        self.recommendations = None # List of tuples (track_id, energy, similarity, has been recommended)

    def make_recommendations(self, user_index, n=100):
        self.user_index = user_index

        user_items = self.interaction_matrix.tocsr()[user_index]


        top_n_recommendations_indexes, top_n_recommendations_scores = self.als_model.recommend(user_index, user_items, N=n, filter_already_liked_items=True)

        for i in range(len(top_n_recommendations_indexes)):
            print(f"Track ID: {self.track_uniques[top_n_recommendations_indexes[i]]}, Similarity: {top_n_recommendations_scores[i]}")


        track_ids = self.track_uniques[top_n_recommendations_indexes].tolist()
        
        df_filtered = self.df_music_info.set_index('track_id').loc[track_ids][['energy']].reset_index()

        self.recommendations = [(track_id, energy, similarity, False) for (track_id, energy), similarity in zip(df_filtered.itertuples(index=False, name=None), top_n_recommendations_scores)]
        return self.recommendations

    
    def recommend_song(self, energy, energy_margin=0.05):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        
        closest_track_index = None
        distance_to_energy = float('inf')

        for i, (track_id, track_energy, similarity, has_been_recommended) in enumerate(self.recommendations):
            distance = abs(track_energy - energy)

            if not has_been_recommended and distance <= energy_margin:
                self.recommendations[i] = (track_id, track_energy, similarity, True)
                return (track_id, track_energy)
            
            if not has_been_recommended and distance < distance_to_energy:
                closest_track_index = i
                distance_to_energy = distance
        
        if closest_track_index is not None:
            track_id, track_energy, _, _= self.recommendations[closest_track_index]
            self.recommendations[closest_track_index] = (track_id, track_energy, similarity, True)
            return (track_id, track_energy)

        raise ValueError("All recommendations have already been recommended")


    def get_recommendations(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return self.recommendations


    def get_recommendations_ids(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return [track_id for track_id, _, _, _ in self.recommendations]
    
    def get_recommendations_info(self):
        track_ids = [track_id for track_id, _, _, _ in self.recommendations]
        df_ordered = self.df_music_info.set_index('track_id').loc[track_ids].reset_index()
        return df_ordered

### Example

In [103]:
user_index = 0 #User for wich recommendations will be made
user_listened_songs = df_users[df_users['user_id'] == user_uniques[user_index]].track_id
df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

,track_id,name,artist,energy
79,TRAAHSY128F147BB5C,Speed of Sound,Coldplay,0.8980
796,TRDLMWP128F426BF6C,Ragged Wood,Fleet Foxes,0.6850
1158,TRPUGUW128F426BF6F,He Doesn't Know Why,Fleet Foxes,0.5580
1743,TRRYCBO128F932A2C7,Love Shack,The B-52's,0.9020
2350,TRFUPBA128F934F7E1,Paper Gangsta,Lady Gaga,0.5300
2585,TRVODUZ128F934D094,All That We Perceive,Thievery Corporation,0.8340
2722,TRHDDQG12903CB53EE,Heaven's Gonna Burn Your Eyes,Thievery Corporation,0.3370
5416,TRTUCUY128F92E1D24,Stacked Actors,Foo Fighters,0.9340
5428,TRLXSNR128F429361D,Generator,Foo Fighters,0.9240
5819,TRADVZX128F426BF79,Sun Giant,Fleet Foxes,0.0626


In [105]:
als_model_paper = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20, num_threads=-1, random_state=RANDOM_STATE)
als_model_paper.fit(interaction_matrix_user_item_paper)

  0%|          | 0/20 [00:00<?, ?it/s]

In [133]:
als_recommender = ALSRecommender(interaction_matrix_user_item, track_uniques, df_music_info, als_model=als_model_paper)
als_recommender.make_recommendations(0, n=20)
als_recommender.get_recommendations_info()

Track ID: TRXWAZC128F9314B3E, Similarity: 0.9041305184364319
Track ID: TRONYHY128F92C9D11, Similarity: 0.8991858959197998
Track ID: TRPFYYL128F92F7144, Similarity: 0.7460957169532776
Track ID: TRRXGAK128EF349F1A, Similarity: 0.7222431302070618
Track ID: TRFTUIW128E0784B9F, Similarity: 0.7117827534675598
Track ID: TRAEUIW12903D018F0, Similarity: 0.6926310658454895
Track ID: TROWQGL128E0784BAA, Similarity: 0.6922769546508789
Track ID: TRZNAHL128F9327D5A, Similarity: 0.6821429133415222
Track ID: TRWAUCC128F428E6FD, Similarity: 0.6802917718887329
Track ID: TRCQYJE128E078FA49, Similarity: 0.6755713820457458
Track ID: TRXDINI128F146B94D, Similarity: 0.652403712272644
Track ID: TRGGLQD12903D1256C, Similarity: 0.6478930115699768
Track ID: TROMKCG128F9320C09, Similarity: 0.6346142888069153
Track ID: TREQOIZ128F42BCE96, Similarity: 0.629111647605896
Track ID: TRLXFKL128F426BF75, Similarity: 0.626681923866272
Track ID: TRQEMRN128F933B4B2, Similarity: 0.6162332892417908
Track ID: TRJQRAP128F9314B3

,track_id,name,artist,energy
0,TRXWAZC128F9314B3E,Heartbreak Warfare,John Mayer,0.5720
1,TRONYHY128F92C9D11,Revelry,Kings of Leon,0.7480
2,TRPFYYL128F92F7144,Float On,Modest Mouse,0.8930
3,TRRXGAK128EF349F1A,The Way I Are,Timbaland,0.9460
4,TRFTUIW128E0784B9F,Bubble Toes,Jack Johnson,0.5320
5,TRAEUIW12903D018F0,You And Your Heart,Jack Johnson,0.5520
6,TROWQGL128E0784BAA,Taylor,Jack Johnson,0.7250
7,TRZNAHL128F9327D5A,Gears,Miss May I,0.9750
8,TRWAUCC128F428E6FD,Staple It Together,Jack Johnson,0.7940
9,TRCQYJE128E078FA49,The News,Jack Johnson,0.0818


In [131]:
als_recommender.get_recommendations()

[('TRXWAZC128F9314B3E', 0.572, np.float32(0.9041305), False),
 ('TRONYHY128F92C9D11', 0.748, np.float32(0.8991859), False),
 ('TRPFYYL128F92F7144', 0.893, np.float32(0.7460957), False),
 ('TRRXGAK128EF349F1A', 0.946, np.float32(0.72224313), False),
 ('TRFTUIW128E0784B9F', 0.532, np.float32(0.71178275), False),
 ('TRAEUIW12903D018F0', 0.552, np.float32(0.69263107), False),
 ('TROWQGL128E0784BAA', 0.725, np.float32(0.69227695), False),
 ('TRZNAHL128F9327D5A', 0.975, np.float32(0.6821429), False),
 ('TRWAUCC128F428E6FD', 0.794, np.float32(0.6802918), False),
 ('TRCQYJE128E078FA49', 0.0818, np.float32(0.6755714), False),
 ('TRXDINI128F146B94D', 0.613, np.float32(0.6524037), False),
 ('TRGGLQD12903D1256C', 0.267, np.float32(0.647893), False),
 ('TROMKCG128F9320C09', 0.866, np.float32(0.6346143), False),
 ('TREQOIZ128F42BCE96', 0.342, np.float32(0.62911165), False),
 ('TRLXFKL128F426BF75', 0.0635, np.float32(0.6266819), False),
 ('TRQEMRN128F933B4B2', 0.275, np.float32(0.6162333), False),
 (

In [107]:
recommended_song = als_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
2122,TRPKQEL128E0795771,In the Waiting Line,Zero 7,0.376


In [108]:
als_model_bm25 = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20, num_threads=-1, random_state=RANDOM_STATE)
als_model_bm25.fit(interaction_matrix_user_item_bm25)

  0%|          | 0/20 [00:00<?, ?it/s]

In [109]:
als_recommender_bm25 = ALSRecommender(interaction_matrix_user_item, track_uniques, df_music_info, als_model=als_model_bm25)
als_recommender_bm25.make_recommendations(0, n=20)
als_recommender_bm25.get_recommendations_info()

,track_id,name,artist,energy
0,TRUJOHU128F424E6A6,White Winter Hymnal,Fleet Foxes,0.4970
1,TRQEMRN128F933B4B2,Sun It Rises,Fleet Foxes,0.2750
2,TRGEIDA128F933B4B8,Tiger Mountain Peasant Song,Fleet Foxes,0.1460
3,TRVRIBZ128F426BF71,Your Protector,Fleet Foxes,0.5370
4,TRLXFKL128F426BF75,Oliver James,Fleet Foxes,0.0635
5,TRUFTBY128F93450B8,Alejandro,Lady Gaga,0.8000
6,TRKABMK128F426BF72,Meadowlarks,Fleet Foxes,0.1660
7,TRDJRES128F933B4BA,Quiet Houses,Fleet Foxes,0.6680
8,TRXWAZC128F9314B3E,Heartbreak Warfare,John Mayer,0.5720
9,TRKOHVA128F426BF70,Heard Them Stirring,Fleet Foxes,0.4700


In [111]:
recommended_song = als_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
16769,TREQOIZ128F42BCE96,I'm Gonna Find Another You,John Mayer,0.342


### Experiments

#### Confidence value proposed in article vs bm25

In [124]:
random_generator = np.random.default_rng(seed=RANDOM_STATE)
random_users = random_generator.choice(np.arange(len(user_uniques)), size=20, replace=False)

first_different_recommendation = []

for rand_user in random_users:
    als_recommender = ALSRecommender(interaction_matrix_user_item, track_uniques, df_music_info, als_model=als_model_paper)
    als_recommender.make_recommendations(rand_user, n=200)
    paper_recommendations = als_recommender.get_recommendations_ids()

    als_recommender = ALSRecommender(interaction_matrix_user_item, track_uniques, df_music_info, als_model=als_model_bm25)
    als_recommender.make_recommendations(rand_user, n=200)
    bm25_recommendations = als_recommender.get_recommendations_ids()

    for i in range(len(paper_recommendations)):
        if paper_recommendations[i] != bm25_recommendations[i]:
            first_different_recommendation.append(i)
            break
    else:
        first_different_recommendation.append(-1)

print("First different recommendation for each user: (-1 = every recommendation is the same)")
print(first_different_recommendation)

First different recommendation for each user: (-1 = every recommendation is the same)
[0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [125]:
analized_user = random_users[0]

als_recommender = ALSRecommender(interaction_matrix_user_item, track_uniques, df_music_info, als_model=als_model_paper)
als_recommender.make_recommendations(analized_user, n=200)
paper_recommendations = als_recommender.get_recommendations_info()

als_recommender = ALSRecommender(interaction_matrix_user_item, track_uniques, df_music_info, als_model=als_model_bm25)
als_recommender.make_recommendations(analized_user, n=200)
bm25_recommendations = als_recommender.get_recommendations_info()

In [126]:
paper_recommendations.head(20)

,track_id,name,artist,energy
0,TRLVQME128F931BAF3,Vanilla Twilight,Owl City,0.523
1,TRJZRTR128F9314B42,Perfectly Lonely,John Mayer,0.530
2,TRVSJOM12903CD2DC1,One Less Lonely Girl,Justin Bieber,0.750
3,TRDGUPO128F9332CF6,Down To Earth,Bungle,0.904
4,TRSUSWW128F93463BF,Imma Be,Black Eyed Peas,0.539
5,TRBNYBX128F422EC61,Hey There Delilah,Plain White T's,0.261
6,TRGCHLH12903CB7352,Party In The U.S.A.,The Barden Bellas,0.360
7,TRCPXID128F92D5D3C,Halo,Depeche Mode,0.508
8,TRYEGSH12903CD2DCE,Overboard,Justin Bieber,0.790
9,TRTKLFX12903CD2DC2,First Dance,Justin Bieber,0.520


In [127]:
bm25_recommendations.head(20)

,track_id,name,artist,energy
0,TRVSJOM12903CD2DC1,One Less Lonely Girl,Justin Bieber,0.750
1,TRCKWGF12903CD2DCD,Never Let You Go,Third Eye Blind,0.940
2,TRDGUPO128F9332CF6,Down To Earth,Bungle,0.904
3,TRYEGSH12903CD2DCE,Overboard,Justin Bieber,0.790
4,TRZEDRT12903CD2DCC,Runaway Love,Justin Bieber,0.633
5,TRTKLFX12903CD2DC2,First Dance,Justin Bieber,0.520
6,TRLVQME128F931BAF3,Vanilla Twilight,Owl City,0.523
7,TRZYAGJ128F9332CEF,Favorite Girl,Justin Bieber,0.681
8,TROMKCG128F9320C09,Uprising,Sabaton,0.866
9,TRJZRTR128F9314B42,Perfectly Lonely,John Mayer,0.530


In [128]:
paper_ids = set(paper_recommendations['track_id'].head(50))
bm25_ids = set(bm25_recommendations['track_id'].head(50))
porcentaje = len(paper_ids & bm25_ids) / len(paper_ids) * 100
print(f"Percentage of paper recommendations that are in bm25 recomendations: {porcentaje:.2f}%")

Percentage of paper recommendations that are in bm25 recomendations: 66.00%


In [129]:
percentage_overlap_20 = []
percentage_overlap_50 = []
percentage_overlap_200 = []

for rand_user in random_users:
    als_recommender = ALSRecommender(interaction_matrix_user_item, track_uniques, df_music_info, als_model=als_model_paper)
    als_recommender.make_recommendations(rand_user, n=200)
    paper_recommendations = als_recommender.get_recommendations_info()

    als_recommender = ALSRecommender(interaction_matrix_user_item, track_uniques, df_music_info, als_model=als_model_bm25)
    als_recommender.make_recommendations(rand_user, n=200)
    bm25_recommendations = als_recommender.get_recommendations_info()

    paper_ids_20 = set(paper_recommendations['track_id'].head(20))
    bm25_ids_20 = set(bm25_recommendations['track_id'].head(20))

    paper_ids_50 = set(paper_recommendations['track_id'].head(50))
    bm25_ids_50 = set(bm25_recommendations['track_id'].head(50))

    paper_ids_200 = set(paper_recommendations['track_id'])
    bm25_ids_200 = set(bm25_recommendations['track_id'])


    overlap = round(len(paper_ids_20 & bm25_ids_20) / len(paper_ids_20) * 100, 2)
    percentage_overlap_20.append(overlap)

    overlap = round(len(paper_ids_50 & bm25_ids_50) / len(paper_ids_50) * 100, 2)
    percentage_overlap_50.append(overlap)

    overlap = round(len(paper_ids_200 & bm25_ids_200) / len(paper_ids_200) * 100, 2)
    percentage_overlap_200.append(overlap)

print("Percentage of overlap between paper and bm25 recommendations for each user:")
print(f"First 20 recommendations: {percentage_overlap_20}")
print(f"First 50 recommendations: {percentage_overlap_50}")
print(f"First 200 recommendations: {percentage_overlap_200}")

print(f"Average percentage of overlap for 20 first recommendations: {np.mean(percentage_overlap_20):.2f}%")
print(f"Median percentage of overlap for 20 first recommendations: {np.median(percentage_overlap_20):.2f}%")

print(f"Average percentage of overlap for 50 first recommendations: {np.mean(percentage_overlap_50):.2f}%")
print(f"Median percentage of overlap for 50 first recommendations: {np.median(percentage_overlap_50):.2f}%")

print(f"Average percentage of overlap for 200 first recommendations: {np.mean(percentage_overlap_200):.2f}%")
print(f"Median percentage of overlap for 200 first recommendations: {np.median(percentage_overlap_200):.2f}%")

Percentage of overlap between paper and bm25 recommendations for each user:
First 20 recommendations: [65.0, 55.0, 60.0, 75.0, 90.0, 80.0, 45.0, 60.0, 70.0, 75.0, 25.0, 50.0, 40.0, 35.0, 25.0, 30.0, 10.0, 80.0, 65.0, 15.0]
First 50 recommendations: [66.0, 46.0, 62.0, 80.0, 82.0, 94.0, 80.0, 58.0, 72.0, 72.0, 28.0, 54.0, 46.0, 32.0, 36.0, 70.0, 6.0, 64.0, 64.0, 22.0]
First 200 recommendations: [46.5, 45.5, 53.5, 65.0, 51.5, 62.0, 78.0, 54.5, 77.5, 50.5, 40.0, 38.0, 44.5, 38.0, 28.0, 62.0, 20.5, 49.0, 42.5, 30.5]
Average percentage of overlap for 20 first recommendations: 52.50%
Median percentage of overlap for 20 first recommendations: 57.50%
Average percentage of overlap for 50 first recommendations: 56.70%
Median percentage of overlap for 50 first recommendations: 63.00%
Average percentage of overlap for 200 first recommendations: 48.88%
Median percentage of overlap for 200 first recommendations: 47.75%
